# Import dependencies

In [15]:
import pandas as pd
import requests
import os
import json
from typing import List
from util.webscraper import WebScraper
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import string

https://python.langchain.com/v0.2/docs/tutorials/rag/

# Data Collection

First, we get the data from the API. As the API is not yet published, both the API-Url and the query to get information on edition-software need to be specified in your .env file. (consult the README for more information)

In [16]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [17]:
# get api_url and query
api_url = os.environ['API_URL']
query = os.environ['QUERY']

# get data from api
api_response = requests.get(api_url + query)

Now that we got the data from the API, we can load it into a dataframe to prepare it to be used as a knowledge base for rag. 

In [18]:
edition_software_info = json.loads(api_response.text)
edition_software_info = pd.DataFrame(edition_software_info)
edition_software_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                31 non-null     object
 1   slug              31 non-null     object
 2   brand_name        31 non-null     object
 3   concept_doi       0 non-null      object
 4   description       28 non-null     object
 5   description_url   3 non-null      object
 6   description_type  31 non-null     object
 7   get_started_url   30 non-null     object
 8   image_id          23 non-null     object
 9   is_published      31 non-null     bool  
 10  short_statement   31 non-null     object
 11  created_at        31 non-null     object
 12  updated_at        31 non-null     object
 13  closed_source     31 non-null     bool  
dtypes: bool(2), object(12)
memory usage: 3.1+ KB


A brief inspection allows us to formulate some initial tasks and questions for this experiment.

- **Preprocessing:** As we can see, not a single entry contains a associated concept_doi. We might consider dropping the column.
- **Impact of using short descriptions only:** Three entries are missing the in depth description. We can assume that rag won't be too useful for these entries. 
- **Impact of additional information:** Only three have a description-url. Down the road, we need to evaluate, if adding info from this source improves the performance of the rag-system.

# Data Cleaning

### 1. Remove Artefacts

Both the `description` and `short_statement` columns seem to be of particular interest for the task at hand. To asses necessary preprocessing step, we'll need to take a closer look at them.

In [19]:
descriptions = edition_software_info[["description", "short_statement"]]
with pd.option_context('display.max_colwidth', None):
    display(descriptions.head())

,description,short_statement
0,"# Erkennen, Transkribieren und Durchsuchen von historischen Dokumenten mitttels KI\n\n- Trainieren von spezifischen Texterkennungsmodellen, die in der Lage sind, handschriftliche, maschinengeschriebene oder gedruckte Dokumente zu erkennen.\n\n- KI-gestützte Erkennung von handgeschriebenem Text, Layout-Analyse und Strukturerkennung.\n\n- Manuelles Transkribieren im Transkriptionseditor\nKI-gestützten Erkennung mittels öffentlicher oder selbst trainierter KI-Modelle\n\n- Durchsuchen von Dokumenten mit erweiterten Suchoptionen, wie z. B. dem Tool zum Aufspüren von Schlüsselwörtern.\n\n\n- Gemeinsames Arbeiten an Dokumenten, Organisation in Sammlungen\n\n- Teilen von Dokumenten durch eine read&search Website oder Export als PDF oder ALTO (XML).\n\n- Alle Transkribus-Inhalte, d.h. hochgeladene Bilder, erkannte Texte, trainierte Erkennungsmodelle und eingegebene Metadaten, werden innerhalb der EU gehostet und sind GDPR konform.","Transkribus ist eine umfassende Plattform für die Digitalisierung, Texterkennung mithilfe Künstlicher Intelligenz, Transkription und das Durchsuchen von historischen Dokumenten."
1,"autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users.\n\nSpecial features of the service include the ability to upload content in tabular format (tsv files) and the ability to publish posts as a thread. In addition to these basic functionalities, more features will be developed in the future.\n\nautodone replaces autoChirp, which offered the same functionality for Twitter before the Twitter API and Twitter itself was massively restricted regarding free and ethical usage.\n\n(quoted from: https://autodone.idh.uni-koeln.de/about, 19.04.2024)\n\n--- \n## Official Site:\n[https://autodone.idh.uni-koeln.de/](https://autodone.idh.uni-koeln.de/)\n\n---\n## Usage Instructions\n[https://autodone.idh.uni-koeln.de/usage](https://autodone.idh.uni-koeln.de/usage)\n","Autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users."
2,"[CollateX](http://collatex.net/) is a software to\n\n 1. read **multiple (≥ 2) versions of a text**, splitting each version into parts (tokens) to be compared,\n 1. **identify similarities of and differences between the versions** (including moved/transposed segments) by aligning tokens, and\n 1. output the alignment results in a **variety of formats for further processing**, for instance\n 1. to support **the production of a critical apparatus** or the stemmatical analysis of a text's genesis.\n\nIt resembles software used to compute differences between files (e.g. [diff](http://en.wikipedia.org/wiki/Diff)) or tools for [sequence alignment](http://en.wikipedia.org/wiki/Sequence_alignment) which are commonly used in Bioinformatics. While CollateX shares some of the techniques and algorithms with those tools, it mainly aims for a flexible and configurable approach to the problem of finding similarities and differences in texts, sometimes trading computational soundness or complexity for the user's ability to influence results.\n\nAs such it is primarily designed for use cases in disciplines like [Philology](http://en.wikipedia.org/wiki/Philology) or – more specifically – the field of [Textual Criticism](http://en.wikipedia.org/wiki/Textual_criticism) where the assessment of findings is based on interpretation and therefore can be supported by computational means but is not necessarily computable.\n\nPlease go to <http://collatex.net/> for further information.","CollateX is a software to (a.) read multiple versions of a text, (b.) identify differences by aligning tokens, and (

As we can see, the `description` column contains some formatting artefacts like `\n` and markdown syntax like `**` and `#`. Let's clean them up.
While we're at it, we can also remove double whitespaces etc.

In [20]:
pattern = '\\n+'
edition_software_info["description_clean"] = edition_software_info["description"].str.replace(pattern, ' ', regex=True)

pattern = r'[*#]+|\s-+\s|]]' #\[\]()<>
edition_software_info["description_clean"] = edition_software_info["description_clean"].str.replace(pattern, ' ', regex=True)

with pd.option_context('display.max_colwidth', None):
    display(edition_software_info[["brand_name", "description", "description_clean"]].head())

,brand_name,description,description_clean
0,Transkribus,"# Erkennen, Transkribieren und Durchsuchen von historischen Dokumenten mitttels KI\n\n- Trainieren von spezifischen Texterkennungsmodellen, die in der Lage sind, handschriftliche, maschinengeschriebene oder gedruckte Dokumente zu erkennen.\n\n- KI-gestützte Erkennung von handgeschriebenem Text, Layout-Analyse und Strukturerkennung.\n\n- Manuelles Transkribieren im Transkriptionseditor\nKI-gestützten Erkennung mittels öffentlicher oder selbst trainierter KI-Modelle\n\n- Durchsuchen von Dokumenten mit erweiterten Suchoptionen, wie z. B. dem Tool zum Aufspüren von Schlüsselwörtern.\n\n\n- Gemeinsames Arbeiten an Dokumenten, Organisation in Sammlungen\n\n- Teilen von Dokumenten durch eine read&search Website oder Export als PDF oder ALTO (XML).\n\n- Alle Transkribus-Inhalte, d.h. hochgeladene Bilder, erkannte Texte, trainierte Erkennungsmodelle und eingegebene Metadaten, werden innerhalb der EU gehostet und sind GDPR konform.","Erkennen, Transkribieren und Durchsuchen von historischen Dokumenten mitttels KI Trainieren von spezifischen Texterkennungsmodellen, die in der Lage sind, handschriftliche, maschinengeschriebene oder gedruckte Dokumente zu erkennen. KI-gestützte Erkennung von handgeschriebenem Text, Layout-Analyse und Strukturerkennung. Manuelles Transkribieren im Transkriptionseditor KI-gestützten Erkennung mittels öffentlicher oder selbst trainierter KI-Modelle Durchsuchen von Dokumenten mit erweiterten Suchoptionen, wie z. B. dem Tool zum Aufspüren von Schlüsselwörtern. Gemeinsames Arbeiten an Dokumenten, Organisation in Sammlungen Teilen von Dokumenten durch eine read&search Website oder Export als PDF oder ALTO (XML). Alle Transkribus-Inhalte, d.h. hochgeladene Bilder, erkannte Texte, trainierte Erkennungsmodelle und eingegebene Metadaten, werden innerhalb der EU gehostet und sind GDPR konform."
1,Autodone,"autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users.\n\nSpecial features of the service include the ability to upload content in tabular format (tsv files) and the ability to publish posts as a thread. In addition to these basic functionalities, more features will be developed in the future.\n\nautodone replaces autoChirp, which offered the same functionality for Twitter before the Twitter API and Twitter itself was massively restricted regarding free and ethical usage.\n\n(quoted from: https://autodone.idh.uni-koeln.de/about, 19.04.2024)\n\n--- \n## Official Site:\n[https://autodone.idh.uni-koeln.de/](https://autodone.idh.uni-koeln.de/)\n\n---\n## Usage Instructions\n[https://autodone.idh.uni-koeln.de/usage](https://autodone.idh.uni-koeln.de/usage)\n","autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users. Special features of the service include the ability to upload content in tabular format (tsv files) and the ability to publish posts as a thread. In addition to these basic functionalities, more features will be developed in the future. autodone replaces autoChirp, which offered the same functionality for Twitter before the Twitter API and Twitter itself was massively restricted regarding free and ethical usage. (quoted from: https://autodone.idh.uni-koeln.de/about, 19.04.2024) Official Site: [https://autodone.idh.uni-koeln.de/](https://autodone.idh.uni-koeln.de/) Usage Instructions [https://autodone.idh.uni-koeln.de/usage](https://autodone.idh.uni-koeln.de/usage)"
2,CollateX,"[CollateX](http://collatex.net/) is a software to\n\n 1. read **multiple (≥ 2) versions of a text**, splitting each version into parts (tokens) to be compared,\n 1. **

### 2. Fill nan values

Before we continue preprocessing the data for later vectorization, we need to check for missing values and replace them with empty strings.

In [21]:
edition_software_info["description_clean"].fillna("", inplace=True)

/var/folders/wv/j42m3p7n5jl51rggmnpn_6940000gn/T/ipykernel_31398/3857300838.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  edition_software_info["description_clean"].fillna("", inplace=True)


# Scrape Webpages

To provide additional context-information for the retrieval process, we'll scrape all webpages referenced in the software-description.

### 1. Get urls

First, we isolate the urls from our description.

In [22]:
pattern = r"((?:https?:\/\/|w{3}.)[\w\d%/.-]+)"

urls = edition_software_info["description"].str.extractall(pattern)
urls = urls.droplevel(1)
urls_grouped = urls.groupby(urls.index).agg((lambda x: ','.join(set(x))))
edition_software_info["urls"] = urls_grouped

with pd.option_context('display.max_colwidth', None):
    display(edition_software_info[["description_clean", "urls"]].head())

,description_clean,urls
0,"Erkennen, Transkribieren und Durchsuchen von historischen Dokumenten mitttels KI Trainieren von spezifischen Texterkennungsmodellen, die in der Lage sind, handschriftliche, maschinengeschriebene oder gedruckte Dokumente zu erkennen. KI-gestützte Erkennung von handgeschriebenem Text, Layout-Analyse und Strukturerkennung. Manuelles Transkribieren im Transkriptionseditor KI-gestützten Erkennung mittels öffentlicher oder selbst trainierter KI-Modelle Durchsuchen von Dokumenten mit erweiterten Suchoptionen, wie z. B. dem Tool zum Aufspüren von Schlüsselwörtern. Gemeinsames Arbeiten an Dokumenten, Organisation in Sammlungen Teilen von Dokumenten durch eine read&search Website oder Export als PDF oder ALTO (XML). Alle Transkribus-Inhalte, d.h. hochgeladene Bilder, erkannte Texte, trainierte Erkennungsmodelle und eingegebene Metadaten, werden innerhalb der EU gehostet und sind GDPR konform.",NaN
1,"autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users. Special features of the service include the ability to upload content in tabular format (tsv files) and the ability to publish posts as a thread. In addition to these basic functionalities, more features will be developed in the future. autodone replaces autoChirp, which offered the same functionality for Twitter before the Twitter API and Twitter itself was massively restricted regarding free and ethical usage. (quoted from: https://autodone.idh.uni-koeln.de/about, 19.04.2024) Official Site: [https://autodone.idh.uni-koeln.de/](https://autodone.idh.uni-koeln.de/) Usage Instructions [https://autodone.idh.uni-koeln.de/usage](https://autodone.idh.uni-koeln.de/usage)","https://autodone.idh.uni-koeln.de/usage,https://autodone.idh.uni-koeln.de/,https://autodone.idh.uni-koeln.de/about"
2,"[CollateX](http://collatex.net/) is a software to 1. read multiple (≥ 2) versions of a text , splitting each version into parts (tokens) to be compared, 1. identify similarities of and differences between the versions (including moved/transposed segments) by aligning tokens, and 1. output the alignment results in a variety of formats for further processing , for instance 1. to support the production of a critical apparatus or the stemmatical analysis of a text's genesis. It resembles software used to compute differences between files (e.g. [diff](http://en.wikipedia.org/wiki/Diff)) or tools for [sequence alignment](http://en.wikipedia.org/wiki/Sequence_alignment) which are commonly used in Bioinformatics. While CollateX shares some of the techniques and algorithms with those tools, it mainly aims for a flexible and configurable approach to the problem of finding similarities and differences in texts, sometimes trading computational soundness or complexity for the user's ability to influence results. As such it is primarily designed for use cases in disciplines like [Philology](http://en.wikipedia.org/wiki/Philology) or – more specifically – the field of [Textual Criticism](http://en.wikipedia.org/wiki/Textual_criticism) where the assessment of findings is based on interpretation and therefore can be supported by computational means but is not necessarily computable. Please go to <http://collatex.net/> for further information.","http://en.wikipedia.org/wiki/Philology,http://en.wikipedia.org/wiki/Textual_criticism,http://en.wikipedia.org/wiki/Sequence_alignment,http://collatex.net/,http://en.wikipedia.org/wiki/Diff"
3,"Der Mathematiker Donald E. Knuth entwickelte Ende der Siebziger Jahre ein Textsatzprogramm, um seine Bücher schöner setzen zu können. Das so entstandene TeX-System verbreitete sich recht schnell, erforderte aber eine intensive Einarbeitung in die zugehörige Programmiersprache. Mit LaTeX 2e, dem Anfang der Neunziger Jahre entwickelten Makropaket",NaN
4,,NaN


### 2. Scrape Webpages

Now we scrape the paragraphs from the webpages we found. 
The webscraper will take the list of urls associated with an entry and will save paragraphs from all webpages as a string in a column of our dataframe. 

**This might take some time**

In [23]:
webscraper = WebScraper(tags = ["p"], exclude = ["wikipedia"])
edition_software_info["webpages_text"] = edition_software_info["urls"].apply(lambda x: webscraper.scrape(x))

Scraping https://autodone.idh.uni-koeln.de/usage with parameters tags = ['p']
Scraping https://autodone.idh.uni-koeln.de/ with parameters tags = ['p']
Scraping https://autodone.idh.uni-koeln.de/about with parameters tags = ['p']
Scraping http://collatex.net/ with parameters tags = ['p']
Scraping http://www.tei-c.org/ with parameters tags = ['p']
Scraping http://vbd.humnet.unipi.it/ with parameters tags = ['p']
Request exception occurred for http://vbd.humnet.unipi.it/: HTTPConnectionPool(host='vbd.humnet.unipi.it', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x146ead640>: Failed to establish a new connection: [Errno 61] Connection refused'))
Scraping https://sites.fastspring.com/stefanhagel/product/cte with parameters tags = ['p']
Scraping https://www.oeaw.ac.at/oesterreichische-akademie-der-wissenschaften with parameters tags = ['p']
Scraping https://opensource.org/licenses/EUPL-1.2 with parameters tags = ['p']

### 3. Inspect data

In [24]:
edition_software_info[["urls", "webpages_text"]].head()

,urls,webpages_text
0,NaN,NaN
1,"https://autodone.idh.uni-koeln.de/usage,https:...",On this page you will find instructions on how...
2,"http://en.wikipedia.org/wiki/Philology,http://...","“In a language, in the system of language, the..."
3,NaN,NaN
4,NaN,NaN


Now that the data is collected from the webpages, we can take a look at the average length of the texts received for each entry.

In [25]:
length = edition_software_info["webpages_text"].apply(lambda x: len(x) if not pd.isna(x) else 0)
length[length>0].describe()

count       13.000000
mean     14854.923077
std      17207.648297
min        640.000000
25%       1919.000000
50%       6739.000000
75%      23517.000000
max      55063.000000
Name: webpages_text, dtype: float64

Looking only at entries, that we were able to collected webpage text for, we have an average character count of about 15.000 per entry. 
The standard deviation is quite large compared to the mean, indicating that there is a high degree of variability in character counts.

The distribution is skewed towards entries with lower character counts, while some outliers with a high character counts pull the mean upwards.



# Export dataset

In [26]:
current_dir = os.getcwd()
path = os.path.join(current_dir, 'data/edition_software_info.csv')
edition_software_info.to_csv(path)

# Preprocessing

Now, we can preprocess the newfound text using the function we defined earlier. Again, we have to replace missing values with empty strings.

First, let us reimport the data.

In [29]:
current_dir = os.getcwd()
path = os.path.join(current_dir, "data/edition_software_info.csv")
edition_software_info = pd.read_csv(path)
edition_software_info[["description", "description_clean", "webpages_text", "urls"]] = edition_software_info[["description", "description_clean", "webpages_text", "urls"]].fillna('')

### 2. Remove links

First, we'll remove all links from the descriptions.

In [30]:
pattern = r"((?:https?:\/\/|w{3}.)[\w\d%/.-]+)"
edition_software_info["description_preprocessed"] = edition_software_info["description_clean"].str.replace(pattern, '', regex=True)
edition_software_info["description_preprocessed"].head(3)

0      Erkennen, Transkribieren und Durchsuchen von...
1    autodone is a service for the automated, time-...
2    [CollateX]() is a software to  1. read  multip...
Name: description_preprocessed, dtype: object

### 1. Remove Stopwords and Punctuation

For the later vectorisation of the texts, we remove both german and english stopwords.

In [31]:
def preprocess(stopwords: List[str], text: str) -> str:
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))    
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

In [32]:
# get stopwords
stopwords_english = set(stopwords.words('english'))
stopwords_german = set(stopwords.words('german'))
stopwords_combined = stopwords_german.union(stopwords_english)

edition_software_info["description_preprocessed"] = edition_software_info["description_preprocessed"].apply(lambda x: preprocess(stopwords_combined, x))
edition_software_info["description_preprocessed"].head()

0    erkennen transkribieren durchsuchen historisch...
1    autodone service automated timecontrolled publ...
2    collatex software 1 read multiple ≥ 2 versions...
3    mathematiker donald e knuth entwickelte ende s...
4                                                     
Name: description_preprocessed, dtype: object

In [33]:
edition_software_info["webpages_text_preprocessed"] = edition_software_info["webpages_text"].apply(lambda x: preprocess(stopwords_combined, x) if not pd.isna(x) else "")
edition_software_info["webpages_text_preprocessed"].head()

0                                                     
1    page find instructions use autodone based scre...
2    “in language system language differences”– jac...
3                                                     
4                                                     
Name: webpages_text_preprocessed, dtype: object

# Vectorization 1: TFIDF

We'll start off with a simple tf-idf vectorisation. 

### 1: Fit TF-IDF Vectorizer
First, we fit the vectorizer on the preprocessed descriptions. 
This way, the vectorizer can transform text into numerical feature vectors based on the learned vocabulary and its distribution over documents.

In [36]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_matrix = tfidf_vectorizer.fit_transform(edition_software_info["description_preprocessed"])

# display the resulting matrix
tfidf_matrix_beautify = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_matrix_beautify.head()

,1514,1514 digital,19042024,19042024 official,1905,1905 1931,1931,1931 used,1986,1986 authors,...,österreichischen,österreichischen akademie,úfal,úfal institute,überführt,überführt neben,überschaubaren,überschaubaren teil,überwiegende,überwiegende teil
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.089367,0.089367,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2: Inspect the tf-idf representations

Each column in this dataframe is a unique word, while each row is a document. The cells denote the number of occurances of a word in a document, weighted by the words potential to be distinctive.

Let's take a look at the tf-idf filtered words for each description (You can find them in the column "tf_idf_filtered_words")

In [37]:
# Define the threshold for TF-IDF scores
threshold = 0.2

# Filter words with TF-IDF scores greater than the threshold for each document
def filter_words_by_threshold(row, threshold):
    filtered_words = [(word, score) for word, score in zip(tfidf_matrix_beautify.columns, row) if score > threshold]
    return sorted(filtered_words, key=lambda x: x[1], reverse=True)


# Apply the function to each row of the TF-IDF DataFrame
filtered_words = tfidf_matrix_beautify.apply(lambda row: filter_words_by_threshold(row, threshold), axis=1)

# Create a dataframe to display the filtered words
filtered_words_df = pd.DataFrame(filtered_words, columns=["tf_idf_filtered_words"])
tfidf_display = pd.concat([edition_software_info["description_clean"], filtered_words_df], axis=1)
    
with pd.option_context('display.max_colwidth', None):
    display(tfidf_display[["description_clean", "tf_idf_filtered_words"]].head(5))

,description_clean,tf_idf_filtered_words
0,"Erkennen, Transkribieren und Durchsuchen von historischen Dokumenten mitttels KI Trainieren von spezifischen Texterkennungsmodellen, die in der Lage sind, handschriftliche, maschinengeschriebene oder gedruckte Dokumente zu erkennen. KI-gestützte Erkennung von handgeschriebenem Text, Layout-Analyse und Strukturerkennung. Manuelles Transkribieren im Transkriptionseditor KI-gestützten Erkennung mittels öffentlicher oder selbst trainierter KI-Modelle Durchsuchen von Dokumenten mit erweiterten Suchoptionen, wie z. B. dem Tool zum Aufspüren von Schlüsselwörtern. Gemeinsames Arbeiten an Dokumenten, Organisation in Sammlungen Teilen von Dokumenten durch eine read&search Website oder Export als PDF oder ALTO (XML). Alle Transkribus-Inhalte, d.h. hochgeladene Bilder, erkannte Texte, trainierte Erkennungsmodelle und eingegebene Metadaten, werden innerhalb der EU gehostet und sind GDPR konform.","[(dokumenten, 0.33021410932225603)]"
1,"autodone is a service for the automated, time-controlled publication of status updates on any Mastodon instance. The codebase is developed under a free license by the Department of Digital Humanities at the University of Cologne and is open to all interested users. Special features of the service include the ability to upload content in tabular format (tsv files) and the ability to publish posts as a thread. In addition to these basic functionalities, more features will be developed in the future. autodone replaces autoChirp, which offered the same functionality for Twitter before the Twitter API and Twitter itself was massively restricted regarding free and ethical usage. (quoted from: https://autodone.idh.uni-koeln.de/about, 19.04.2024) Official Site: [https://autodone.idh.uni-koeln.de/](https://autodone.idh.uni-koeln.de/) Usage Instructions [https://autodone.idh.uni-koeln.de/usage](https://autodone.idh.uni-koeln.de/usage)","[(twitter, 0.26810009116835964)]"
2,"[CollateX](http://collatex.net/) is a software to 1. read multiple (≥ 2) versions of a text , splitting each version into parts (tokens) to be compared, 1. identify similarities of and differences between the versions (including moved/transposed segments) by aligning tokens, and 1. output the alignment results in a variety of formats for further processing , for instance 1. to support the production of a critical apparatus or the stemmatical analysis of a text's genesis. It resembles software used to compute differences between files (e.g. [diff](http://en.wikipedia.org/wiki/Diff)) or tools for [sequence alignment](http://en.wikipedia.org/wiki/Sequence_alignment) which are commonly used in Bioinformatics. While CollateX shares some of the techniques and algorithms with those tools, it mainly aims for a flexible and configurable approach to the problem of finding similarities and differences in texts, sometimes trading computational soundness or complexity for the user's ability to influence results. As such it is primarily designed for use cases in disciplines like [Philology](http://en.wikipedia.org/wiki/Philology) or – more specifically – the field of [Textual Criticism](http://en.wikipedia.org/wiki/Textual_criticism) where the assessment of findings is based on interpretation and therefore can be supported by computational means but is not necessarily computable. Please go to <http://collatex.net/> for further information.","[(differences, 0.2150540011707874)]"
3,"Der Mathematiker Donald E. Knuth entwickelte Ende der Siebziger Jahre ein Textsatzprogramm, um seine Bücher schöner setzen zu können. Das so entstandene TeX-System verbreitete sich recht schnell, erforderte aber eine intensive Einarbeitung in die zugehörige Programmiersprache. Mit LaTeX 2e, dem Anfang der Neunziger Jahre entwickelten Makropaket","[(jahre, 0.26585686887909327)]"
4,,[]


### 3. Test tfidf-representation

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

query = 'I am looking for a transcription tool'
# Preprocess the query
query = preprocess(stopwords_combined, query)
# Transform the query to TF-IDF space
query_tfidf = tfidf_vectorizer.transform([query]) 

# Compute cosine similarity between the query and the documents
similarities = cosine_similarity(query_tfidf, tfidf_matrix)

similarity_df = pd.DataFrame({
    'similarity_score': similarities[0]
})

result_df = pd.concat([edition_software_info[["description_clean"]], similarity_df], axis=1)
result_df_sorted = result_df.sort_values(by='similarity_score', ascending=False)

# print the top 3 description, that might be relevant to our query
with pd.option_context('display.max_colwidth', None):
    display(result_df_sorted[['description_clean', 'similarity_score']].head(3))


description_clean  \
22  [Transcribo](https://tcdh.uni-trier.de/en/projekt/transcribo) is an editing tool developed by the Trier Center for Digital Humanities as part of the project “Arthur Schnitzler: Digital Historical-Critical Edition”. The digital tool can support users in transcribing texts in various fields. In addition to the actual transcription, the texts can also be annotated and text-genetic facts can be marked up. Transcribo offers the possibility to transcribe texts productively and in a time-saving way, using the same intuitive approach as you are used to from manual transcription. The tool offers all the subtleties needed for a differentiated transcription and supports the working process both efficiently and easily comprehensible.   Transcribe almost like with sheet and pencil Transcribing is done in a graphical editor directly on the facsimile, simulating the procedure of analog transcribing with sheet and pencil. This intuitive approach is an advantage especially when transcribing difficult to decipher manuscripts or heavily revised typescript pages and additionally reduces the learning time for the tool, since no markup language (e.g. TEI/XML) has to be learned. Another advantage of this method is that by working on the facsimile, a positional link between the transcribed text and the image is automatically stored. This positional data can be used in the context of the electronic presentation to visualize the link between text and facsimile for the user in a sustainable way, e.g. in the form of cross-fades or mouse-over effects.   Can be used both locally and with FuD  Transcribo can be used both locally and in conjunction with our FuD system.  In local operation, facsimiles located on the terminal device can be opened and edited. All processed information is stored locally in the form of XML files. When FuD is used in addition, the data is stored in FuD's own relational database, which enables collaborative work. In addition, some functionalities, such as the recording of cross-page phenomena, the correction tool or the storage of text states, are only available in this operating mode. The many years of development work accompanying the project have paid off.  Thus, adaptations and ideas from numerous humanities projects at TCDH have been incorporated into the work and have resulted in a very differentiated and practically tested tool.    Some of the most important features of Transcribo 1.  Fine-grained markup of microgenetic facts  (at document, page, sentence/line, word, and graph levels) 2.  Markup of complex interrelated phenomena/change operations  via relations (page and cross-page) 3.  Correction function  for automated comparison of A and B files from different users to ensure error-free transcription and annotation ('double-blind process') 4.  OCR functionality  for automatic reading of typescripts or prints (Tesseract) 5.  Navigation perspective:  interface to the FuD database (linking facsimiles with metadata recorded in FuD) 6.  Structural perspective:  overview of a text section of any size (synopsis of different graphical representations: Facsimile, transcription, annotations) and possibility of depositing cross-page phenomena. 7.  Module for defining and labeling text states or text layers.    Technical requirements The application was implemented in the Eclipse integrated development environment. It is a rich client application that reuses parts of the development environment and was developed in the Java programming language. We currently provide a version for Windows and macOS operating systems.   Projects using Transcribo [Arthur Schnitzler Digital](https://tcdh.uni-trier.de/en/projekt/arthur-schnitzler-digital) [The Augsburg Master Builder’s Ledgers](https://tcdh.uni-trier.de/en/projekt/augsburg-master-builders-ledgers) [Digitale Edition and Analysis of the Medulla Gestorum Treverensium by Johann Enen (1514)](https://tcdh.uni-trier.de/en/projekt/digitale-edition-and-analysis-medulla-ges